In [1]:
!cd /kaggle/working/

In [2]:
!git clone https://github.com/Trusted-AI/adversarial-robustness-toolbox.git
!cd adversarial-robustness-toolbox
!pip install -e 

Cloning into 'adversarial-robustness-toolbox'...
remote: Enumerating objects: 75424, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 75424 (delta 2), reused 0 (delta 0), pack-reused 75418
Receiving objects: 100% (75424/75424), 408.65 MiB | 26.51 MiB/s, done.
Resolving deltas: 100% (59761/59761), done.

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [3]:
!git clone -b main https://github.com/hammaad2002/ARTToolbox.git

Cloning into 'ARTToolbox'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 343 (delta 24), reused 27 (delta 11), pack-reused 301
Receiving objects: 100% (343/343), 628.20 KiB | 10.47 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [4]:
my_string = '/kaggle/working/'

In [5]:
import os,shutil
os.remove(my_string + "adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/imperceptible_asr_pytorch.py")
shutil.copy(my_string + 'ARTToolbox/imperceptible_asr_pytorch.py', my_string+'adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/')
os.remove(my_string + "adversarial-robustness-toolbox/art/estimators/speech_recognition/pytorch_deep_speech.py")
shutil.copy(my_string + 'ARTToolbox/pytorch_deep_speech.py',my_string+'adversarial-robustness-toolbox/art/estimators/speech_recognition/')
shutil.copy(my_string + 'ARTToolbox/wav2vec2ModelWrapper.py',my_string+'adversarial-robustness-toolbox/art/estimators/speech_recognition/')
os.remove(my_string + "adversarial-robustness-toolbox/art/attacks/evasion/adversarial_asr.py")
shutil.copy(my_string + 'ARTToolbox/adversarial_asr.py',my_string+'adversarial-robustness-toolbox/art/attacks/evasion/')
os.remove(my_string + "adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/imperceptible_asr.py")
shutil.copy(my_string + 'ARTToolbox/imperceptible_asr.py', my_string+'adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/')

'/kaggle/working/adversarial-robustness-toolbox/art/attacks/evasion/imperceptible_asr/imperceptible_asr.py'

In [6]:
import sys
sys.path.append(my_string + "adversarial-robustness-toolbox")

In [7]:
!cd /kaggle/working/

In [8]:
!git clone -b main https://github.com/hammaad2002/CRDNN_Model.git

Cloning into 'CRDNN_Model'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 130 (delta 8), reused 9 (delta 2), pack-reused 106
Receiving objects: 100% (130/130), 48.94 MiB | 21.39 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [9]:
import IPython.display as ipd

# **MODEL WRAPPER**

In [10]:
# Import PyTorchSpeechRecognizer
import logging
from typing import TYPE_CHECKING, List, Optional, Tuple, Union
import numpy as np
from art.estimators.pytorch import PyTorchEstimator
from art.estimators.speech_recognition.speech_recognizer import SpeechRecognizerMixin, PytorchSpeechRecognizerMixin
import torch.nn.functional as F
import torch
# Import wav2vec2.0 model from pytorch models
from pkg_resources import packaging  
from art import config
from art.utils import get_file
import torchaudio
from art.estimators.estimator import BaseEstimator, LossGradientsMixin, NeuralNetworkMixin

if TYPE_CHECKING:
    # pylint: disable=C0412
    import torch

    from art.defences.postprocessor.postprocessor import Postprocessor
    from art.defences.preprocessor.preprocessor import Preprocessor
    from art.utils import CLIP_VALUES_TYPE, PREPROCESSING_TYPE

logger = logging.getLogger(__name__)

In [11]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

In [12]:
class wav2vec2Model(PytorchSpeechRecognizerMixin, SpeechRecognizerMixin, PyTorchEstimator):
    
    # Initialize your wrapper with your model and other parameter
    estimator_params = PyTorchEstimator.estimator_params + ["optimizer", "use_amp", "opt_level", "lm_config", "verbose"] + BaseEstimator.estimator_params+ NeuralNetworkMixin.estimator_params+ ["device_type"]
    
    #constructor method
    def __init__(
        self,
        modell,
        optimizer: Optional["torch.optim.Optimizer"] = None,
        use_amp: bool = False,
        opt_level: str = "O1",
        verbose: bool = True,
        clip_values: Optional["CLIP_VALUES_TYPE"] = None,
        preprocessing_defences: Union["Preprocessor",List["Preprocessor"],None]=None,
        postprocessing_defences: Union["Postprocessor",List["Postprocessor"],None]=None,
        preprocessing: "PREPROCESSING_TYPE" = None,

    ):

        #initializing the parent class's contructors
        super().__init__(
            model = modell,
            clip_values=clip_values,
            channels_first=None,
            preprocessing_defences=preprocessing_defences,
            postprocessing_defences=postprocessing_defences,
            preprocessing=preprocessing,
        )

        #class attributes
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.__model = modell
        #self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self._input_shape = None
        self._use_amp = use_amp
        self._opt_level = opt_level
        
    #transcription encoder for CTC LOSS
    def encode_transcription(self, transcription):
        # Define the dictionary
        dictionary = {'-': 0, '|': 1, 'E': 2, 'T': 3, 'A': 4, 'O': 5, 'N': 6, 'I': 7, 'H': 8, 'S': 9, 'R': 10, 'D': 11, 'L': 12, 'U': 13, 'M': 14, 'W': 15, 'C': 16, 'F': 17, 'G': 18, 'Y': 19, 'P': 20, 'B': 21, 'V': 22, 'K': 23, "'": 24, 'X': 25, 'J': 26, 'Q': 27, 'Z': 28}

        # Convert transcription string to list of characters
        chars = list(transcription)

        # Encode each character using the dictionary
        encoded_chars = [dictionary[char] for char in chars]

        # Concatenate the encoded characters to form the final encoded transcription
        encoded_transcription = torch.tensor(encoded_chars)

        #returning the encoded transcription
        return encoded_transcription

    
    def loss_gradient(self, x: np.ndarray, y: np.ndarray, device = "cuda", **kwargs) -> np.ndarray:
        """
        Compute the gradient of the loss function w.r.t. `x`.
        :param x: Samples of shape (nb_samples, seq_length). Note that, it is allowable that sequences in the batch
                  could have different lengths. A possible example of `x` could be:
                  `x = np.array([np.array([0.1, 0.2, 0.1, 0.4]), np.array([0.3, 0.1])])`.
        :param y: Target values of shape (nb_samples). Each sample in `y` is a string and it may possess different
                  lengths. A possible example of `y` could be: `y = np.array(['SIXTY ONE', 'HELLO'])`.
        :return: Loss gradients of the same shape as `x`.
        """
        #importing required modules
        import torch

        #Making sure all data is on same device
        x = torch.from_numpy(x)
        x = x.to(device)
        audioo = x.clone()
        
        # Performing inference
        self.__model.cuda()
        self.__model.eval()

        audioo.requires_grad = True
        
        # Encoding the target transcription
        encoded_transcription = self.encode_transcription(y[0]) #.replace(" ","|")
        
        # Forward pass the data through the model
        emission, _ = self.__model(audioo)

        # Declaring arguments for CTC Loss
        targets = torch.tensor(encoded_transcription, dtype=torch.long)
        output_sizes = torch.tensor([emission.shape[1]], dtype=torch.long)
        emission = emission.transpose(0, 1)
        target_sizes = torch.tensor([len(encoded_transcription)], dtype=torch.long)

        # Calculating loss
        loss = F.ctc_loss(emission, targets, output_sizes, target_sizes)
        
        # Zero all existing gradients (new)
        self.__model.zero_grad()
        
        # Calculate gradients of model in backward pass
        loss.backward()
        
        # Collect datagrad
        data_grad = audioo.grad.data
        
        # Detaching the output
        data_grad.detach_()
        
        '''with torch.no_grad():
            temptemp = audioo.grad.to(device)
            temptemp = temptemp.detach_()'''
        
        # Normalizing the gradients to range [-1, 1]
        data_grad = data_grad / (data_grad.abs().max() + 1e-12)
     
        return data_grad.cpu().numpy()
    
    # Implement compute_loss_and_decoded_output method
    def compute_loss_and_decoded_output(
        self, masked_adv_input: "torch.Tensor", original_output: np.ndarray, **kwargs
    ) -> Tuple["torch.Tensor", np.ndarray]:
        
        """
        Compute loss function and decoded output.
        :param masked_adv_input: The perturbed inputs.
        :param original_output: Target values of shape (nb_samples). Each sample in `original_output` is a string and
                                it may possess different lengths. A possible example of `original_output` could be:
                                `original_output = np.array(['SIXTY ONE'])`
        :param real_lengths: Real lengths of original sequences.
        :return: The loss and the decoded output.
        """
        
        #importing required modules
        import torch
        import torchaudio
        
        #importing the wav2vec2 model's dictionary
        bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        
        # Changing the variable name for convenience 
        x_tensor = masked_adv_input.to(self._device)
        x_tensor = x_tensor.float()
        
        # Performing inference
        self.__model.cuda()
        self.__model.eval()
        
        x_tensor.requires_grad = True
        
        # Forward pass the data through the model
        emission, _ = self.__model(x_tensor)

        # Decoding the model's output
        decoder = GreedyCTCDecoder(labels = bundle.get_labels())
        transcript = decoder(emission[0])
        #transcript = transcript.replace("|"," ")
        
        # Encoding the target transcription
        encoded_transcription = self.encode_transcription(original_output[0]) #.replace(" ","|")
        
        # Declaring arguments for CTC Loss
        targets = torch.tensor(encoded_transcription, dtype=torch.long)
        output_sizes = torch.tensor([emission.shape[1]], dtype=torch.long)
        emission = emission.transpose(0, 1)
        target_sizes = torch.tensor([len(encoded_transcription)], dtype=torch.long)

        # Calculating loss
        loss = F.ctc_loss(emission, targets, output_sizes, target_sizes)
        
        # Zero all existing gradients (new)
        self.__model.zero_grad()
        
        #returning loss and decoded transcription
        return loss, np.array([transcript])

    def predict(
        self, x: np.ndarray, batch_size: int = 1, **kwargs
    ) -> Union[Tuple[np.ndarray, np.ndarray], np.ndarray]:

        #importing required modules
        import torch
        import torchaudio

        #importing the wav2vec2 model's dictionary
        bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        
        # Changing the variable name for convenience
        x = torch.Tensor(x)
        x_tensor = x.to(self._device)
        x_tensor = x_tensor.float()
        
        # Performing inference
        self.__model.cuda()
        self.__model.eval()
        
        # Forward pass the data through the model
        emission, _ = self.__model(x_tensor)

        # Decoding the model's output
        decoder = GreedyCTCDecoder(labels = bundle.get_labels())
        transcript = decoder(emission[0])
        #transcript = transcript.replace("|"," ")
        
        #returning the decoded output of model which is the 'prediction'
        return np.array([transcript])

    # Implement to_training_mode method 
    def to_training_mode(self) -> None:
      
        # Set your model to training mode 
        self.__model.train()

    # Implement sample_rate property 
    @property
    def sample_rate(self) -> int:
      
        # Return the sample rate of your model (you may need to check the documentation or source code of your model)
        return 16000
    
    @property
    def input_shape(self) -> Tuple[int, ...]:
        """
        Return the shape of one input sample.
        :return: Shape of one input sample.
        """
        return (1, self._input_length)
    
    @property
    def model(self):
        """
        Get current model.

        :return: Current model.
        """
        return self.__model
    
    @property
    def device(self) -> "torch.device":
        """
        Get current used device.

        :return: Current used device.
        """
        return self._device
    
    @property
    def use_amp(self) -> bool:
        """
        Return a boolean indicating whether to use the automatic mixed precision tool.

        :return: Whether to use the automatic mixed precision tool.
        """
        return self._use_amp  

    @property
    def opt_level(self) -> str:
        """
        Return a string specifying a pure or mixed precision optimization level.

        :return: A string specifying a pure or mixed precision optimization level. Possible
                 values are `O0`, `O1`, `O2`, and `O3`.
        """
        return self._opt_level 
    
    def get_activations(
        self, x: np.ndarray, layer: Union[int, str], batch_size: int, framework: bool = False
    ) -> np.ndarray:
        print("It is being called somewhere just for checking.")
        pass

# **------ATTACK IMPLEMENTATION ON DEEPSPEECH MODEL USING ART TOOLBOX------**

# **----------------------ATTACK PREPROCESSING AND INITIALIZATION----------------------**

In [13]:
import torch
import torchaudio
#from art.estimators.speech_recognition.wav2vec2ModelWrapper import wav2vec2Model
from art.attacks.evasion import FastGradientMethod , BasicIterativeMethod, ProjectedGradientDescent, ImperceptibleASRPyTorch, CarliniWagnerASR, ImperceptibleASR
import numpy as np
import librosa

# **WAV2VEC2.0 MODEL DOWNLOADING**

In [14]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

# **LOADING THE AUDIO**

In [15]:
# Load some audio data for testing
x, sr = librosa.load( my_string + "CRDNN_Model/AudioSamplesASR/spk1_snt1.wav" , sr=None )
x = torch.from_numpy( x )
x = x.unsqueeze( 0 ) 
x = x.numpy()
print( x.shape )
# y1 =  np.array(['THE CHILD ALMOST HURT THE BIG DOG'.replace(" ","|")])
# print(y1.shape)

(1, 45920)


In [16]:
# print(y1)

# **INITIALIZING THE WRAPPER FOR ART TOOLBOX**

In [17]:
estimator = wav2vec2Model(model)

# **TARGET TRANSCRIPTION**

In [18]:
y1 =  np.array(['THE MAN ALMOST KILL THE BIG CAT'.replace(" ","|")]) #lumber one target transcription
print(y1.shape) 

(1,)


# **TRIED HYPERPARAMETER SEARCH FOR ADVERSARIAL ATTACKS (CURRENT PROGRESS = NO SUCCESS)**

In [19]:
# from datasets import load_metric
# %%capture 
# !pip install jiwer
# wer_metric = load_metric("wer")
# class CarliniWagnerASRWrapper:
#     def __init__(self, estimator, batch_size=1):
#         self.estimator = estimator
#         self.batch_size = batch_size
        
#     def fit(self, X, y, **kwargs):
#         eps = kwargs.get('eps', 0.0015)
#         max_iter = kwargs.get('max_iter', 1000)
#         learning_rate = kwargs.get('learning_rate', 50.0)
#         decrease_factor_eps = kwargs.get('decrease_factor_eps', 1.0)
#         num_iter_decrease_eps = kwargs.get('num_iter_decrease_eps', 1)
        
#         attack = CarliniWagnerASR(
#             estimator=self.estimator, 
#             batch_size=self.batch_size, 
#             eps=eps, 
#             max_iter=max_iter, 
#             learning_rate=learning_rate, 
#             decrease_factor_eps=decrease_factor_eps, 
#             num_iter_decrease_eps=num_iter_decrease_eps
#         )
        
#         X_adv = attack.generate(x=X, y=y)
        
#         return X_adv, y
    
#     def get_params(self, deep=True):
#         return {'model': self.estimator}
    
#     def set_params(self, **params):
#         if 'model' in params:
#             self.model = params['model']
#         return self
    
#     def score(self, true, pred):
#         wer = wer_metric.compute(predictions=pred_str, references=label_str)
#         return {"wer": wer}
#     from sklearn.model_selection import RandomizedSearchCV

# # Define the hyperparameters and their ranges to search
# param_dist = {
#     'eps': [0.001, 0.0015, 0.002],
#     'max_iter': [500, 1000, 1500],
#     'learning_rate': [10.0, 50.0, 100.0],
#     'decrease_factor_eps': [0.5, 1.0, 2.0],
#     'num_iter_decrease_eps': [10, 20, 30]
# }

# # Create a RandomizedSearchCV object with the attack object and hyperparameters to search
# attack_wrapper = CarliniWagnerASRWrapper(estimator=estimator, batch_size=1)
# random_search = RandomizedSearchCV(attack_wrapper, param_distributions=param_dist, n_iter=10, cv=3)

# # Fit the RandomizedSearchCV object to the data to find the best hyperparameters
# random_search.fit(x, y1)

# # Print the best hyperparameters and success rate found by the RandomizedSearchCV object
# print('Best hyperparameters:', random_search.best_params_)
# print('Best success rate:', random_search.best_score_)

# **------------------------------FAST GRADIENT DESCENT ATTACK-----------------------------**

# **PARAMETERS:**

estimator – A trained classifier.

norm – The norm of the adversarial perturbation. Possible values: “inf”, np.inf, 1 or 2.

eps – Attack step size (input variation).

eps_step – Step size of input variation for minimal perturbation computation.

targeted (bool) – Indicates whether the attack is targeted (True) or untargeted (False)

num_random_init (int) – Number of random initialisations within the epsilon ball. For random_init=0 starting at the original input.

batch_size (int) – Size of the batch on which adversarial samples are generated.

minimal (bool) – Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation.

summary_writer – Activate summary writer for TensorBoard. Default is False and deactivated summary writer. If True save runs/CURRENT_DATETIME_HOSTNAME in current directory. If of type str save in path. If of type SummaryWriter apply provided custom summary writer. Use hierarchical folder structure to compare between runs easily. e.g. pass in ‘runs/exp1’, ‘runs/exp2’, etc. for each new experiment to compare across them.

In [20]:
# Creating adversarial attack (FGSM) instance
attack1 = FastGradientMethod(estimator=estimator, batch_size= 1, eps = 0.5, targeted= True, norm = 2, eps_step = 10, num_random_init = 0)
# Generate an adversarial example from the original audio data
x_adv1 = attack1.generate(x=x, y=y1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [21]:
# Compare the transcription of the original and adversarial audio data
y_pred1 = estimator.predict( x )
y_adv_pred1 = estimator.predict( x_adv1 )
print("Original transcription:    ", y_pred1[0].replace("|"," ") )
print("Adversarial transcription: ", y_adv_pred1[0].replace("|"," ") )
print("Targetted Transcription:   ", y1[0].replace("|"," ") )

Original transcription:     HE CHILD ALMOST HURT THE SMALL DOG 
Adversarial transcription:  THE CHILD ALMOST HHURT THE SMALL DOG 
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [22]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [23]:
ipd.Audio(x_adv1, rate = sr)

# **-----------------------------BASIC ITERATIVE METHOD ATTACK-----------------------------**

# **PARAMETERS:**
estimator – A trained classifier.

eps – Maximum perturbation that the attacker can introduce.

eps_step – Attack step size (input variation) at each iteration.

max_iter (int) – The maximum number of iterations.

targeted (bool) – Indicates whether the attack is targeted (True) or untargeted (False).

batch_size (int) – Size of the batch on which adversarial samples are generated.

verbose (bool) – Show progress bars.

In [24]:
# Creating adversarial attack (BIM) instance
attack2 = BasicIterativeMethod(estimator=estimator, batch_size= 1, eps = 0.0002, targeted= True, eps_step= 0.00001, max_iter= 100, verbose= True)
# Generate an adversarial example from the original audio data
x_adv2 = attack2.generate(x=x, y=y1)

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [25]:
# Compare the transcription of the original and adversarial audio data
y_pred2 = estimator.predict( x )
y_adv_pred2 = estimator.predict( x_adv2 )
print("Original transcription:    ", y_pred2[0].replace("|"," ") )
print("Adversarial transcription: ", y_adv_pred2[0].replace("|"," ") )
print("Targetted Transcription:   ", y1[0].replace("|"," ") )

Original transcription:     HE CHILD ALMOST HURT THE SMALL DOG 
Adversarial transcription:  THE CHILD ALMOST HURT THE SMALL DOG 
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [26]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [27]:
ipd.Audio(x_adv2, rate = sr)

# **--------------------------PROJECTED GRADIENT DESCENT ATTACK-------------------------**

# **PARAMETERS:**
estimator – An trained estimator.

norm – The norm of the adversarial perturbation supporting “inf”, np.inf, 1 or 2.

eps – Maximum perturbation that the attacker can introduce.

eps_step – Attack step size (input variation) at each iteration.

random_eps (bool) – When True, epsilon is drawn randomly from truncated normal distribution. The literature suggests this for FGSM based training to generalize across different epsilons. eps_step is modified to preserve the ratio of eps / eps_step. The effectiveness of this method with PGD is untested (https://arxiv.org/pdf/1611.01236.pdf).

decay – Decay factor for accumulating the velocity vector when using momentum.

max_iter (int) – The maximum number of iterations.

targeted (bool) – Indicates whether the attack is targeted (True) or untargeted (False).

num_random_init (int) – Number of random initialisations within the epsilon ball. For num_random_init=0 starting at the original input.

batch_size (int) – Size of the batch on which adversarial samples are generated.

summary_writer – Activate summary writer for TensorBoard. Default is False and deactivated summary writer. If True save runs/CURRENT_DATETIME_HOSTNAME in current directory. If of type str save in path. If of type SummaryWriter apply provided custom summary writer. Use hierarchical folder structure to compare between runs easily. e.g. pass in ‘runs/exp1’, ‘runs/exp2’, etc. for each new experiment to compare across them.

verbose (bool) – Show progress bars.

BTW THERE IS ALSO ANOTHER IMPLEMENTATION OF PGD GIVEN IN DOCUMENTATION AS : classart.attacks.evasion.ProjectedGradientDescent

AND THE DOCUMENTATION CAN BE ACCESSED THROUGH THIS LINK: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/attacks/evasion.html#projected-gradient-descent-pgd-pytorch

In [28]:
# Creating adversarial attack (PGD) instance
attack3 = ProjectedGradientDescent(estimator=estimator, batch_size= 1, eps = 0.5, targeted= True, eps_step= 0.0055, max_iter= 200, verbose = True, norm = 2)
# Generate an adversarial example from the original audio data
x_adv3 = attack3.generate(x=x, y=y1)

PGD - Iterations:   0%|          | 0/200 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [29]:
# Compare the transcription of the original and adversarial audio data
y_pred3 = estimator.predict( x )
y_adv_pred3 = estimator.predict( x_adv3 )
print("Original transcription:    ", y_pred3[0].replace("|"," ") )
print("Adversarial transcription: ", y_adv_pred3[0].replace("|"," ") )
print("Targetted Transcription:   ", y1[0].replace("|"," ") )

Original transcription:     HE CHILD ALMOST HURT THE SMALL DOG 
Adversarial transcription:  MR
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [30]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [31]:
ipd.Audio(x_adv3, rate = sr)

# **--------------------------------CARLINI AND WAGNER ATTACK-------------------------------**

# **PARAMETERS:**
estimator – A trained speech recognition estimator.

eps (float) – Initial max norm bound for adversarial perturbation.

learning_rate (float) – Learning rate of attack.

max_iter (int) – Number of iterations.

decrease_factor_eps (float) – Decrease factor for epsilon (Paper default: 0.8).

num_iter_decrease_eps (int) – Iterations after which to decrease epsilon if attack succeeds (Paper default: 10).

batch_size (int) – Batch size.

In [32]:
# Creating adversarial attack (CW) instance
attack4 = CarliniWagnerASR(estimator=estimator, batch_size= 1, eps = 0.0035, max_iter = 1000, learning_rate = 50.0, decrease_factor_eps= 1.0, num_iter_decrease_eps= 1)
# Generate an adversarial example from the original audio data
x_adv4 = attack4.generate(x=x, y=y1)

100%|██████████| 1/1 [01:06<00:00, 66.64s/it]


In [33]:
# Compare the transcription of the original and adversarial audio data
y_pred4 = estimator.predict( x )
y_adv_pred4 = estimator.predict( x_adv4 )
print("Original transcription:    ", y_pred4[0].replace("|"," ") )
print("Adversarial transcription: ", y_adv_pred4[0].replace("|"," ") )
print("Targetted Transcription:   ", y1[0].replace("|"," ") )

Original transcription:     HE CHILD ALMOST HURT THE SMALL DOG 
Adversarial transcription:  HE CHILD MST  HURTTHE SMILE DOG 
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [34]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [35]:
ipd.Audio(x_adv4, rate = sr)

# **---------------------------------IMPERCEPTIBLE ASR ATTACK--------------------------------**

# **PARAMETERS:**
estimator – A trained speech recognition estimator.

masker – A Psychoacoustic masker.

eps (float) – Initial max norm bound for adversarial perturbation.

learning_rate_1 (float) – Learning rate for stage 1 of attack.

max_iter_1 (int) – Number of iterations for stage 1 of attack.

alpha (float) – Initial alpha value for balancing stage 2 loss.

learning_rate_2 (float) – Learning rate for stage 2 of attack.

max_iter_2 (int) – Number of iterations for stage 2 of attack.

loss_theta_min (float) – If imperceptible loss reaches minimum, stop early. Works best with batch_size=1.

decrease_factor_eps (float) – Decrease factor for epsilon (Paper default: 0.8).

num_iter_decrease_eps (int) – Iterations after which to decrease epsilon if attack succeeds (Paper default: 10).

increase_factor_alpha (float) – Increase factor for alpha (Paper default: 1.2).

num_iter_increase_alpha (int) – Iterations after which to increase alpha if attack succeeds (Paper default: 20).

decrease_factor_alpha (float) – Decrease factor for alpha (Paper default: 0.8).

num_iter_decrease_alpha (int) – Iterations after which to decrease alpha if attack fails (Paper default: 50).

batch_size (int) – Batch size.

In [36]:
from art.attacks.evasion.imperceptible_asr.imperceptible_asr import PsychoacousticMasker

In [37]:
masker = PsychoacousticMasker(window_size= 512, hop_size= 160) # need this as a parameter for this attack for hiding noise almost completely

In [38]:
# Creating adversarial attack (IMPERCEPTIBLE ASR attack) instance
attack5 = ImperceptibleASR(estimator=estimator, masker = masker,  batch_size= 1, eps = 0.0000001)
# Generate an adversarial example from the original audio data
x_adv5 = attack5.generate(x=x, y=y1)

  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[-9.1552734e-05 -9.1552734e-05 -9.1552734e-05 ... -5.3710938e-03
 -5.1879883e-03 -5.3710938e-03]


/opt/conda/lib/python3.7/site-packages/torch/functional.py:633: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/SpectralOps.cpp:801.)
  normalized, onesided, return_complex)
100%|██████████| 1/1 [00:49<00:00, 49.95s/it]


In [39]:
# Compare the transcription of the original and adversarial audio data
y_pred5 = estimator.predict( x )
y_adv_pred5 = estimator.predict( x_adv5 )
print("Original transcription:    ", y_pred5[0].replace("|"," ") )
print("Adversarial transcription: ", y_adv_pred5[0].replace("|"," ") )
print("Targetted Transcription:   ", y1[0].replace("|"," ") )

Original transcription:     HE CHILD ALMOST HURT THE SMALL DOG 
Adversarial transcription:  THE CHILD ALMOST HURT  THE SMALL DOG 
Targetted Transcription:    THE MAN ALMOST KILL THE BIG CAT


**Clean Audio**

In [40]:
ipd.Audio(x, rate = sr)

**Perturbed Audio**

In [41]:
ipd.Audio(x_adv5, rate = sr)

# **----------------------------------------------------------------------------------------------------**